# imports

In [1]:
import sys
sys.path.append("C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package")
from data_preparation import DataPreparation, Analysis, multi_data_ops
import data_operations as do

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

import data_transforations as data_tr


In [2]:
path="C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/Data2/"

## Models

In [3]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV, KFold
from scipy.stats import loguniform

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

from sklearn.svm import SVC

from sklearn.neural_network import MLPClassifier

from imblearn.ensemble import BalancedRandomForestClassifier



## Scoring

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

### Functions

In [5]:
def data_prep(data,target,features=[],filter=None,column_excl = ['1month_lapse', '3month_lapse', '6month_lapse', 'ann_lapse', 'lapse_flag', 'policy_name','lead_provider_name','campaign_name','occupation']):

    if filter is not None:
        data = data[data[filter].notnull()].copy()
    # impute missing values
    # impute missing values
    df=DataPreparation(df=data)
    imp_dict={'optionality': 'mode',
              'occupation':'none'}
    df.imputating_nulls(imp_dict=imp_dict)
    data=df.imputed_df.copy()
    #grouping lead-provider_name
    data['lead_provider_groups']= data['lead_provider_name'].apply(lead_prov_grouping)
    data=pd.get_dummies(data,columns=['education', 'gender', 'smoker_status',
       'sold_socio_economic_class', 'occupation_class',
       'lead_provider_groups', 'lead_type', 'sales_channel','hiv_group', 'securitygroup_id', 'age_band',
       'income_band', 'last_benefit_amount_band', 'cover_amount_full_band',
       'total_funeral_premium_band', 'original_premium_band','underwriting_outcome'], dtype=int)
    if features == []:
        cols=set(data.columns)
        features=list(cols.difference(column_excl))
         
    X = data[features].copy()
    Y = data[target].astype(int).copy()
    # Train Test Split
    X_train, X_test, Y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42, stratify=Y)

    return X_train, X_test, Y_train, y_test


# Modelling

In [ ]:
from sklearn.metrics import make_scorer
def my_custom_accuracy(y_true, y_pred):
    w=2
    cm = confusion_matrix(y_true, y_pred)
    true_0 =(cm[0][0]/(cm[0][0]+cm[0][1]))
    true_1= w*(cm[1][1]/(cm[1][1]+cm[1][0]))

    return (true_0+true_1)/(1+w)
custom_scorer = make_scorer(my_custom_accuracy, greater_is_better=True)

Target = ['1month_lapse', '3month_lapse', '6month_lapse', 'ann_lapse', 'lapse_flag']

## Data for modeling

In [19]:
#imp_features=[]
target= ''
#filter=''
data=pd.DataFrame(pd.read_csv(path+"near_ftr_data_eco_6n_v2.csv"))
data=data.drop(columns=['Unnamed: 0'])
#class_weight = {0:1,1:1}


In [7]:
data.head()

,policy_id,policy_name,age,benefits_count,education,gender,income,smoker_status,sold_socio_economic_class,securitygroup_id,...,res_code_other_sum,res_code_payment_sum,res_code_sale_sum,res_code_update_sum,inflation start,unemployment rate start,end_date,inflation end,unemployment rate end,last in 3 m
0,100003c9-2094-3107-e604-65d8e6b4ec03,MWL008050072,51,1.0,Matric,Male,40000.0,S,1.0,MWL,...,0.0,0.0,0.0,0.0,4.5,34.300,1970-01-01 00:00:00.000002025,3.0,33.05,False
1,10000cc8-8f08-5e2c-b6ad-661d48606710,ML24-0199554,23,1.0,Matric + 3 year Diploma,Female,20000.0,N,2.0,MWL,...,0.0,0.0,0.0,0.0,4.4,32.600,1970-01-01 00:00:00.000002025,3.0,33.05,False
2,100307d2-6857-7f50-833c-66b391c37879,ML24-0254853,63,1.0,Matric + 3 year Diploma,Female,45000.0,E,1.0,MWL,...,0.0,3.0,1.0,0.0,4.4,32.600,1970-01-01 00:00:00.000002025,3.0,33.05,False
3,10048ce0-253d-aeff-4f59-64e76756bddf,ML23-0067459,41,1.0,Matric + 3 year Diploma,Male,70000.0,S,1.0,MWL,...,0.0,0.0,0.0,0.0,6.0,32.375,1970-01-01 00:00:00.000002025,3.0,33.05,False
4,10061226-ed6a-b733-47f0-65fb970086cd,MWL007912004,36,5.0,4 year Degree,Female,40000.0,N,1.0,MWL,...,0.0,0.0,0.0,0.0,4.5,34.300,1970-01-01 00:00:00.000002025,3.0,33.05,False


In [20]:
data.drop(columns=['reason', 'policy_type', 'has_been_recaptured'], inplace=True)
data_info = DataPreparation(df=data)

In [21]:
data_info.uniq_df

,no. of unique entries
policy_id,62536
policy_name,62536
age,48
benefits_count,19
education,8
...,...
unemployment rate start,11
end_date,4
inflation end,4
unemployment rate end,4


In [ ]:
data_info.null_df
#impute: 'optionality' with mode, 'orginal_premium' with premium, 'prev_premium' with premium, 'cover_amount_full' with mean, 'n month ago payment' with "NA"
#drop: 'reason', 'policy_type', 'has_been_recaptured'



,nulls,percentage nulls
policy_id,0,0.000000
policy_name,0,0.000000
age,0,0.000000
benefits_count,0,0.000000
education,0,0.000000
...,...,...
unemployment rate start,0,0.000000
end_date,0,0.000000
inflation end,0,0.000000
unemployment rate end,0,0.000000


In [ ]:

data[['1 month ago payment','2 month ago payment','3 month ago payment','4 month ago payment','5 month ago payment','6 month ago payment']]=data[['1 month ago payment','2 month ago payment','3 month ago payment','4 month ago payment','5 month ago payment','6 month ago payment']].fillna('NA')

imp_dict={'optionality': 'mode',
              'cover_amount_full':'mean'}
data_info.imputating_nulls(imp_dict=imp_dict)
data=data_info.imputed_df

In [ ]:
data=data.apply(lambda row: )

In [16]:
data_info.imputed_df

,policy_id,policy_name,age,benefits_count,education,gender,income,smoker_status,sold_socio_economic_class,securitygroup_id,...,res_code_other_sum,res_code_payment_sum,res_code_sale_sum,res_code_update_sum,inflation start,unemployment rate start,end_date,inflation end,unemployment rate end,last in 3 m
0,100003c9-2094-3107-e604-65d8e6b4ec03,MWL008050072,51,1.0,Matric,Male,40000.0,S,1.0,MWL,...,0.0,0.0,0.0,0.0,4.5,34.300,1970-01-01 00:00:00.000002025,3.0,33.05,False
1,10000cc8-8f08-5e2c-b6ad-661d48606710,ML24-0199554,23,1.0,Matric + 3 year Diploma,Female,20000.0,N,2.0,MWL,...,0.0,0.0,0.0,0.0,4.4,32.600,1970-01-01 00:00:00.000002025,3.0,33.05,False
2,100307d2-6857-7f50-833c-66b391c37879,ML24-0254853,63,1.0,Matric + 3 year Diploma,Female,45000.0,E,1.0,MWL,...,0.0,3.0,1.0,0.0,4.4,32.600,1970-01-01 00:00:00.000002025,3.0,33.05,False
3,10048ce0-253d-aeff-4f59-64e76756bddf,ML23-0067459,41,1.0,Matric + 3 year Diploma,Male,70000.0,S,1.0,MWL,...,0.0,0.0,0.0,0.0,6.0,32.375,1970-01-01 00:00:00.000002025,3.0,33.05,False
4,10061226-ed6a-b733-47f0-65fb970086cd,MWL007912004,36,5.0,4 year Degree,Female,40000.0,N,1.0,MWL,...,0.0,0.0,0.0,0.0,4.5,34.300,1970-01-01 00:00:00.000002025,3.0,33.05,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62531,ffaa032e-2a01-cf52-cc1a-65fb3147c393,MWL006676786,51,1.0,Matric + 3 year Diploma,Female,40000.0,N,1.0,MWL,...,0.0,0.0,6.0,0.0,4.2,28.700,1970-01-01 00:00:00.000002025,3.0,33.05,False
62532,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,MWL007292690,41,1.0,Matric,Female,14000.0,N,2.0,MWL,...,0.0,0.0,0.0,0.0,3.3,29.175,1970-01-01 00:00:00.000002025,3.0,33.05,False
62533,ffd20919-303c-e88e-fb1f-65fba1578790,MWL008568948,26,2.0,Matric + 3 year Diploma,Female,14000.0,N,2.0,MWL,...,0.0,0.0,1.0,0.0,6.9,33.500,1970-01-01 00:00:00.000002025,3.0,33.05,False
62534,ffe1059c-beea-354d-27fa-6629fe94a5cf,MWL006194665,45,1.0,3 year Degree,Female,30000.0,N,1.0,MWL,...,0.0,0.0,0.0,0.0,4.7,27.125,1970-01-01 00:00:00.000002025,3.0,33.05,False


In [ ]:
data[target].sum()*100/data[target].count()

8.773518462621993

### Scaling data

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
data_s=data.copy()

In [ ]:
data_s[[ 'eml','pml', 'benefits_count', 'funeral_count', 'duration_policy_start_delay', '# calls',
       '# calls contacted', 'res_code_QA_sum', 'res_code_claims_sum','res_code_no interest_sum', 'res_code_other_sum',
       'res_code_payment_sum', 'res_code_quote_sum', 'res_code_sale_sum','res_code_update_sum', 'inflation', 'unemployment rate']] = scale.fit_transform(data_s[[ 'eml','pml', 'benefits_count', 'funeral_count', 'duration_policy_start_delay', '# calls',
       '# calls contacted', 'res_code_QA_sum', 'res_code_claims_sum','res_code_no interest_sum', 'res_code_other_sum',
       'res_code_payment_sum', 'res_code_quote_sum', 'res_code_sale_sum','res_code_update_sum', 'inflation', 'unemployment rate']])

In [ ]:
X_train,x_test, Y_train, y_test = data_prep(data_s,target,filter=filter)

### try resampling SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy=0.6,random_state=42)
X_smote, Y_smote = sm.fit_resample(X_train.copy(), Y_train.copy())

#X_train=X_smote.copy()
#Y_train=Y_smote.copy()

In [ ]:
Y_smote.sum()*100/Y_smote.count()

37.49569114098587